In [1]:
import pandas as pd
import requests
import calendar
from typing import List

In [24]:
def TrainBetweenStation(source: str, destination: str, pref_running_days: List[str]=[]):
    data = pd.read_csv("Rail_data.txt", low_memory=False)
    Station_Code = data[["Station Name", "Station Code"]]
    Station_Code = Station_Code.drop_duplicates(subset=['Station Name'])
    Station_Code["Station Name"] = Station_Code["Station Name"].apply(lambda x: x.split(" ")[0])
    Station_Code = Station_Code.reset_index()
    scode = Station_Code.loc[Station_Code['Station Name'] == source.upper(), 'Station Code'].iloc[0]
    dcode = Station_Code.loc[Station_Code['Station Name'] == destination.upper(), 'Station Code'].iloc[0]

    url = f"https://indian-railway-api.cyclic.app/trains/betweenStations/?from={scode}&to={dcode}"

    payload={}
    headers = {}
    output = {}
    
    response = requests.request("GET", url, headers=headers, data=payload)

    main = ""
    for i, res in enumerate(response.json()["data"]):
        name = res["train_base"]["train_name"]
        number = res["train_base"]["train_no"]
        ftime = res["train_base"]["from_time"]
        ttime = res["train_base"]["to_time"]
        trtime = res["train_base"]["travel_time"]
        rdays = res["train_base"]["running_days"]
        
        days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
        running_days = []
        
        for j, char in enumerate(rdays):
            if char == '1':
                running_days.append(calendar.day_name[j])
        rdays = ', '.join(running_days)

        if float(ftime[:2])>=12:
            ftime+=" PM"
        else:
            ftime+=" AM"

        if float(ttime[:2])>=12:
            ttime+=" PM"
        else:
            ttime+=" AM"

        if len(pref_running_days) == 0:
            print("Train No. :", number)
            print("Train Name :", name)
            print("Arrival Time :", ftime)
            print("Reaching Time :", ttime)
            print("Duration (in hours) :", trtime)
            print("Running Days :", rdays)
            print("********************")
            output.append()
        else:
            search_set = set(pref_running_days)
            all_found = search_set.intersection(set(running_days))
            if all_found:  
                print("Train No. :", number)
                print("Train Name :", name)
                print("Arrival Time :", ftime)
                print("Reaching Time :", ttime)
                print("Duration (in hours) :", trtime)
                print("Running Days :", rdays)
                print("********************")

    

In [25]:
TrainBetweenStation("kharagpur", "kamakhya", ["Monday", "Thursday"])

Train No. : 12507
Train Name : ARONAI EXPRESS
Arrival Time : 09.13 AM
Reaching Time : 04.40 AM
Duration (in hours) : 19.27
Running Days : Thursday
********************
Train No. : 06569
Train Name : SMVB GHY SPL
Arrival Time : 09.45 AM
Reaching Time : 04.40 AM
Duration (in hours) : 18.55
Running Days : Monday
********************
Train No. : 06521
Train Name : SMVB GHY SPL
Arrival Time : 07.00 AM
Reaching Time : 04.10 AM
Duration (in hours) : 21.10
Running Days : Thursday
********************
Train No. : 22503
Train Name : DBRG VIVEK EXP
Arrival Time : 11.55 AM
Reaching Time : 09.00 AM
Duration (in hours) : 21.05
Running Days : Monday, Wednesday, Thursday, Friday, Saturday
********************
